# Introduction

This notebook is from [Cervix segmentation (GMM) | Kaggle Discussion](https://www.kaggle.com/chattob/cervix-segmentation-gmm/comments/notebook).

But I modified some parts to make it easier use.

This kernels aims at segmenting the cervix using the technique presented in this paper: https://www.researchgate.net/publication/24041301_Automatic_Detection_of_Anatomical_Landmarks_in_Uterine_Cervix_Images

## 1. Basic Operation for Reading Image data

This part shows you how to read image. Of course, this part has been writen as functions.

In [7]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import cv2
import math
from sklearn import mixture
from sklearn.utils import shuffle
from skimage import measure
from glob import glob
import os
from multiprocessing import Pool, cpu_count
from functools import partial
from subprocess import check_output
#print(check_output(["ls", "../input"]).decode("utf8"))
print(check_output(["ls", "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/"]).decode("utf8"))


def get_filename(image_id, image_type):
    """
    Method to get image file path from its id and type   
    """
    if image_type == "Type_1" or \
        image_type == "Type_2" or \
        image_type == "Type_3":
        data_path = os.path.join(TRAIN_DATA, image_type)
    elif image_type == "test":
        data_path = TEST_DATA
    elif image_type == "AType_1" or \
          image_type == "AType_2" or \
          image_type == "AType_3":
        data_path = os.path.join(ADDITIONAL_DATA, image_type)
    else:
        raise Exception("Image type '%s' is not recognized" % image_type)

    ext = 'jpg'
    return os.path.join(data_path, "{}.{}".format(image_id, ext))

def get_image_data(image_id, image_type):
    """
    获取图像以numpy数组的形式，即np.array同时指定确定每张图像的id和类型
    """
    fname = get_filename(image_id, image_type)
    img = cv2.imread(fname)
    #print "get_image_data:"+fname
    assert img is not None, "Failed to read /image : %s, %s" % (image_id, image_type)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

additional_Type_1_v2.7z
additional_Type_2_v2.7z
additional_Type_3_v2.7z
Cervix_types_clasification.pdf
cp_rename_for_same_name_file.sh
others
sample_submission.csv
sample_submission.csv.zip
test
test.7z
test_seg_224.zip
train
train.7z
train_add
train_seg_224
train_seg_224.zip
train_tmp
Type_1
Type_2
Type_3



## 2. Remove Circular Frame

First, we crop the image in order to remove the circular frames that might be present. This is done by finding the largest inscribed rectangle to the thresholded image. The image is then cropped to this rectangle. (see these videos for an explanation of the algorithm: https://www.youtube.com/watch?v=g8bSdXCG-lA, https://www.youtube.com/watch?v=VNbkzsnllsU)

In [8]:
def maxHist(hist):
    maxArea = (0, 0, 0)
    height = []
    position = []
    for i in range(len(hist)):
        if (len(height) == 0):
            if (hist[i] > 0):
                height.append(hist[i])
                position.append(i)
        else: 
            if (hist[i] > height[-1]):
                height.append(hist[i])
                position.append(i)
            elif (hist[i] < height[-1]):
                while (height[-1] > hist[i]):
                    maxHeight = height.pop()
                    area = maxHeight * (i-position[-1])
                    if (area > maxArea[0]):
                        maxArea = (area, position[-1], i)
                    last_position = position.pop()
                    if (len(height) == 0):
                        break
                position.append(last_position)
                if (len(height) == 0):
                    height.append(hist[i])
                elif(height[-1] < hist[i]):
                    height.append(hist[i])
                else:
                    position.pop()    
    while (len(height) > 0):
        maxHeight = height.pop()
        last_position = position.pop()
        area =  maxHeight * (len(hist) - last_position)
        if (area > maxArea[0]):
            maxArea = (area, len(hist), last_position)
    return maxArea
            

def maxRect(img):
    maxArea = (0, 0, 0)
    addMat = np.zeros(img.shape)
    for r in range(img.shape[0]):
        if r == 0:
            addMat[r] = img[r]
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
        else:
            addMat[r] = img[r] + addMat[r-1]
            addMat[r][img[r] == 0] *= 0
            area = maxHist(addMat[r])
            if area[0] > maxArea[0]:
                maxArea = area + (r,)
    return (int(maxArea[3]+1-maxArea[0]/abs(maxArea[1]-maxArea[2])), maxArea[2], maxArea[3], maxArea[1], maxArea[0])

def cropCircle(img):
    if(img.shape[0] > img.shape[1]):
        tile_size = (int(img.shape[1]*256/img.shape[0]),256)
    else:
        tile_size = (256, int(img.shape[0]*256/img.shape[1]))

    img = cv2.resize(img, dsize=tile_size)
            
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY);
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)

    _, contours, _ = cv2.findContours(thresh.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    main_contour = sorted(contours, key = cv2.contourArea, reverse = True)[0]
            
    ff = np.zeros((gray.shape[0],gray.shape[1]), 'uint8') 
    cv2.drawContours(ff, main_contour, -1, 1, 15)
    ff_mask = np.zeros((gray.shape[0]+2,gray.shape[1]+2), 'uint8')
    cv2.floodFill(ff, ff_mask, (int(gray.shape[1]/2), int(gray.shape[0]/2)), 1)
    
    rect = maxRect(ff)
    rectangle = [min(rect[0],rect[2]), max(rect[0],rect[2]), min(rect[1],rect[3]), max(rect[1],rect[3])]
    img_crop = img[rectangle[0]:rectangle[1], rectangle[2]:rectangle[3]]
    cv2.rectangle(ff,(min(rect[1],rect[3]),min(rect[0],rect[2])),(max(rect[1],rect[3]),max(rect[0],rect[2])),3,2)
    
    return [img_crop, rectangle, tile_size]

## 3. Finding Image Center Using Color and Continuous Regions

“For an initial delineation of the cervix, we use two features:  

* the a color channel of the source image in Lab color space (the higher the value of a , the “redder” the pixel color)  
* R, the distance of a pixel from the image center. The R feature provides spatial information and supports the extraction of continuous regions within the image plane."

In [9]:
def Ra_space(img, Ra_ratio, a_threshold):
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = np.zeros((w*h, 2))
    for i in range(w):
        for j in range(h):
            R = math.sqrt((w/2-i)*(w/2-i) + (h/2-j)*(h/2-j))
            Ra[i*h+j, 0] = R
            Ra[i*h+j, 1] = min(imgLab[i][j][1], a_threshold)
            
    Ra[:,0] /= max(Ra[:,0])
    Ra[:,0] *= Ra_ratio
    Ra[:,1] /= max(Ra[:,1])

    return Ra

## 4. Separating Image in New feature Space

"The image is separated next into two clusters in the 2-D (a-R) feature space; we use Gaussian mixture modeling, initialized by a K-means procedure."

In [10]:
def get_and_crop_image(image_id, image_type):
    img = get_image_data(image_id, image_type)
    initial_shape = img.shape
    [img, rectangle_cropCircle, tile_size] = cropCircle(img)
    imgLab = cv2.cvtColor(img, cv2.COLOR_RGB2LAB);
    w = img.shape[0]
    h = img.shape[1]
    Ra = Ra_space(imgLab, 1.0, 150)
    a_channel = np.reshape(Ra[:,1], (w,h))
    
    g = mixture.GaussianMixture(n_components = 2, covariance_type = 'diag', random_state = 0, init_params = 'kmeans')
    image_array_sample = shuffle(Ra, random_state=0)[:1000]
    g.fit(image_array_sample)
    labels = g.predict(Ra)
    labels += 1 # Add 1 to avoid labeling as 0 since regionprops ignores the 0-label.
    
    # The cluster that has the highest a-mean is selected.
    labels_2D = np.reshape(labels, (w,h))
    gg_labels_regions = measure.regionprops(labels_2D, intensity_image = a_channel)
    gg_intensity = [prop.mean_intensity for prop in gg_labels_regions]
    cervix_cluster = gg_intensity.index(max(gg_intensity)) + 1

    mask = np.zeros((w * h,1),'uint8')
    mask[labels==cervix_cluster] = 255
    mask_2D = np.reshape(mask, (w,h))

    cc_labels = measure.label(mask_2D, background=0)
    regions = measure.regionprops(cc_labels)
    areas = [prop.area for prop in regions]

    regions_label = [prop.label for prop in regions]
    largestCC_label = regions_label[areas.index(max(areas))]
    mask_largestCC = np.zeros((w,h),'uint8')
    mask_largestCC[cc_labels==largestCC_label] = 255

    img_masked = img.copy()
    img_masked[mask_largestCC==0] = (0,0,0)
    img_masked_gray = cv2.cvtColor(img_masked, cv2.COLOR_RGB2GRAY);
            
    _,thresh_mask = cv2.threshold(img_masked_gray,0,255,0)
            
    kernel = np.ones((11,11), np.uint8)
    thresh_mask = cv2.dilate(thresh_mask, kernel, iterations = 1)
    thresh_mask = cv2.erode(thresh_mask, kernel, iterations = 2)
    _, contours_mask, _ = cv2.findContours(thresh_mask.copy(),cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)

    main_contour = sorted(contours_mask, key = cv2.contourArea, reverse = True)[0]
    cv2.drawContours(img, main_contour, -1, 255, 3)
    
    x,y,w,h = cv2.boundingRect(main_contour)
    
    rectangle = [x+rectangle_cropCircle[2],
                 y+rectangle_cropCircle[0],
                 w,
                 h,
                 initial_shape[0],
                 initial_shape[1],
                 tile_size[0],
                 tile_size[1]]

    return [image_id, img, rectangle]

In [11]:
def parallelize_image_cropping(image_ids, img_save_base_dir, types, resize_shape=(224, 224)):
    # 新建文件并将crop在原图的位置保存
    out = open('rectangles.csv', "w")
    out.write("image_id,type,x,y,w,h,img_shp_0_init,img_shape1_init,img_shp_0,img_shp_1\n")
    imf_d = {}
    # 利用多线程(python的multiprocessing库)来执行
    p = Pool(cpu_count())
    for type in enumerate(types):
        partial_get_and_crop = partial(get_and_crop_image, image_type = type[1])    
        ret = p.map(partial_get_and_crop, image_ids[type[0]])
        
        # 创建保存新图像的目录
        img_save_type_dir = img_save_base_dir+types[type[0]]+"_seg_"+str(resize_shape[0])+"/"
        if os.path.exists(img_save_type_dir):
            print "dir exists: " + img_save_type_dir
        else:
            print "prepare to create dir:" + img_save_type_dir
            os.mkdir(img_save_type_dir)
            if os.path.exists(img_save_type_dir):
                print "create successed"
            else:
                print "create failed"
                
        for i in xrange(len(ret)):
            out.write(image_ids[type[0]][i])
            out.write(',' + str(type[1]))
            out.write(',' + str(ret[i][2][0]))
            out.write(',' + str(ret[i][2][1]))
            out.write(',' + str(ret[i][2][2]))
            out.write(',' + str(ret[i][2][3]))
            out.write(',' + str(ret[i][2][4]))
            out.write(',' + str(ret[i][2][5]))
            out.write(',' + str(ret[i][2][6]))
            out.write(',' + str(ret[i][2][7]))
            out.write('\n')
            img = get_image_data(image_ids[type[0]][i], type[1])
            if(img.shape[0] > img.shape[1]):
                tile_size = (int(img.shape[1]*256/img.shape[0]), 256)
            else:
                tile_size = (256, int(img.shape[0]*256/img.shape[1]))
            img = cv2.resize(img, dsize=tile_size)
            # 注释掉画出红色框
            # 关于cv2.rectangle：Contour Features — OpenCV-Python Tutorials 1 documentation
            # http://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_imgproc/py_contours/py_contour_features/py_contour_features.html
            # 其中两个参数是对角线坐标
            '''
            cv2.rectangle(img,
                          (ret[i][2][0], ret[i][2][1]), 
                          (ret[i][2][0]+ret[i][2][2], ret[i][2][1]+ret[i][2][3]),
                          255,
                          2)
            '''
            # 重新缩放到指定大小
            img = img[ret[i][2][1]:(ret[i][2][1]+ret[i][2][3]), ret[i][2][0]:(ret[i][2][0]+ret[i][2][2])]
            img = cv2.resize(img, dsize=resize_shape)

            # 打印图像显示
            #plt.imshow(img)            
            #plt.savefig("./"+str(i)+".jpg") # 保存的是带坐标系的图
            img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img_save_dir = img_save_type_dir+ret[i][0]+".jpg"
            cv2.imwrite(img_save_dir, img)
            print str(i+1)+"/"+str(len(ret)), img_save_dir#,type(ret)
            #print ret[i][0]
            #plt.show()
            
        ret = []
    out.close()

    return

## 5. Segmentation on Training Set (给训练集做segmentation)

In [13]:
# plt.imshow
# ?plt.savefig

#print(check_output(["ls", "../input"]).decode("utf8"))
#print(check_output(["ls", "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/"]).decode("utf8"))

# 试验请用如下包含少量图像的目录，否则请注释掉
TRAIN_DATA = "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp"
# 真实训练目录
#TRAIN_DATA = "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train"

save_base_dir = TRAIN_DATA+"/"
resize_shape = (224, 224)
types = ['Type_1', 'Type_2', 'Type_3', ]
type_ids = []

for type in enumerate(types):
    type_i_files = glob(os.path.join(TRAIN_DATA, type[1], "*.jpg"))
    type_i_ids = np.array([s.split('/')[-1].split(".")[0] for s in type_i_files])
    type_ids.append(type_i_ids[:])
    print type, type_i_ids.size

# 打印检查图片是否读取成功
for idx in xrange(len(types)):
    print type_ids[idx][:10]
    print len(type_ids[idx])

# 创建用于保存分割后图像的文件夹

import os

parallelize_image_cropping(type_ids, save_base_dir, types, resize_shape=resize_shape)
    
# 检查是否创建成功
# os.listdir(save_base_dir)

(0, 'Type_1') 3
(1, 'Type_2') 3
(2, 'Type_3') 3
['47' '35' '48']
3
['35' '47' '48']
3
['35' '47' '48']
3
get_image_data:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1/47.jpg
get_image_data:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1/48.jpg
get_image_data:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1/35.jpg
dir exists: /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1_seg_224/
get_image_data:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1/47.jpg
1/3 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1_seg_224/47.jpg
get_image_data:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1/35.jpg
2/3 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1_seg_224/35.jpg
get_image_data:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1/48.jpg
3/3 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp/Type_1_seg_224/48.jpg
get_image_data:/media/yue

## 6. Segmentation on Test Set (给测试集做segmentation)

In [105]:
# plt.imshow
# ?plt.savefig

#print(check_output(["ls", "../input"]).decode("utf8"))
#print(check_output(["ls", "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/"]).decode("utf8"))

TEST_DATA = "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test"
save_base_dir = TEST_DATA+"/"
resize_shape = (224, 224)
types = ['test']
type_ids = []

for type in enumerate(types):
    type_i_files = glob(os.path.join(TEST_DATA[:-4], type[1], "*.jpg"))
    #type_i_ids = np.array(type_i_files[len(TEST_DATA)+8:-4])
    type_i_ids = np.array([s.split('/')[-1].split(".")[0] for s in type_i_files])
    type_ids.append(type_i_ids)

#print type_ids[0]    
#print type_i_files
parallelize_image_cropping(type_ids, save_base_dir, types, resize_shape=resize_shape)

prepare to create dir:/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/
create successed
1/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/0.jpg
2/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/1.jpg
3/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/10.jpg
4/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/100.jpg
5/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/101.jpg
6/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/102.jpg
7/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/103.jpg
8/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/104.jpg
9/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/105.jpg
10/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/106.jpg
11/512 /media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/test/test_seg_224/107.jpg
12/51

## 7. Segmentation on Train-Add Set (给train+add做segmentation)

In [13]:
# plt.imshow
# ?plt.savefig

#print(check_output(["ls", "../input"]).decode("utf8"))
#print(check_output(["ls", "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/"]).decode("utf8"))

# 试验请用如下包含少量图像的目录，否则请注释掉
#TRAIN_DATA = "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_tmp"
# 真实训练目录
TRAIN_DATA = "/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add"

save_base_dir = TRAIN_DATA+"/"
resize_shape = (224, 224)
#types = ['Type_1', 'Type_2', 'Type_3', ]
types = ['Type_2', 'Type_3', ]
type_ids = []

for type in enumerate(types):
    type_i_files = glob(os.path.join(TRAIN_DATA, type[1], "*.jpg"))
    type_i_ids = np.array([s.split('/')[-1].split(".")[0] for s in type_i_files])
    type_ids.append(type_i_ids)
    print type, type_i_ids.size

# 打印检查图片是否读取成功
for idx in xrange(len(types)):
    print type_ids[idx][:10]
    print len(type_ids[idx])
print type_i_files[:10]

# 创建用于保存分割后图像的文件夹

import os

parallelize_image_cropping(type_ids, save_base_dir, types, resize_shape=resize_shape)
    
# 检查是否创建成功
# os.listdir(save_base_dir)

(0, 'Type_2') 4346
(1, 'Type_3') 2426
['0' '1 (2)' '1' '100' '1001' '1002 (2)' '1002' '1005' '1006 (2)' '1006']
4346
['100' '1000' '1003' '1004' '1007' '1008' '1009' '1010' '1012' '1014']
2426
['/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/100.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/1000.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/1003.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/1004.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/1007.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/1008.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/1009.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/1010.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/1012.jpg', '/media/yuens/WIN10-ENTERTENMENT/Kaggle/Intel-CCS/train_add/Type_3/1014.jpg']
prepare to create dir:/medi